<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
   Preprocessing
</div>

## Import thư viện

In [42]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Khám phá dữ liệu

### Đọc file dữ liệu

In [44]:
raw_df = pd.read_csv('survey.csv')
raw_df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


### Dataset có bao nhiêu dòng và cột

In [45]:
raw_df.shape

(1259, 27)

Có 1259 dòng và 27 cột dữ liệu

### Ý nghĩa của từng dòng dữ liệu

- Tập dữ liệu này lấy từ một cuộc khảo sát năm 2014 nhằm đo lường thái độ đối với sức khỏe tâm thần và tần suất rối loạn sức khỏe tâm thần tại nơi làm việc.
- Mỗi dòng là câu trả lời của một người tham gia khảo sát, và thể hiện được các phần như: Thông tin cơ bản của người trả lời (ẩn danh); Thái độ đối với vấn đề sức khỏe tinh thần của chính mình; Cách các công ty, tổ chức cung cấp quyền lợi cho nhân viên có vấn đề sức khỏe tinh thần và thể chất như thế nào;...

### Ý nghĩa các trường dữ liệu

Đọc file `description.csv` trong để hiểu rõ hơn về các cột ta đang làm việc. Lưu vào DataFrame `col_meaning_df` và index chính là `Title` - tên các trường dữ liệu.

In [54]:
pd.set_option('display.max_colwidth', 500)

col_meaning_df = pd.read_csv('description.csv', index_col='Title')
col_meaning_df 

,Meaning
Title,
Timestamp,Thời gian nộp khảo sát
Age,Tuổi
Gender,Giới tính
Country,Quốc gia sinh sống
state,Tên tiểu bang sinh sống
self_employed,Bạn có tự kinh doanh không?
family_history,Gia đình có tiền sử bệnh tâm thần không?
treatment,Bạn đã tìm cách điều trị tình trạng sức khỏe tâm thần chưa?
work_interfere,"Nếu bạn có vấn đề về sức khỏe tâm thần, bạn có cảm thấy nó cản trở công việc của mình không?"


### Xem thử các dòng trong data có bị trùng lặp hay không

In [32]:
duplicated = raw_df.duplicated().any()
duplicated

False

Vậy dữ liệu không có dòng trùng lặp. Ta tiến hành bước tiếp theo

### Loại dữ liệu của mỗi cột là gì? Chúng có phù hợp để xử lí sau này không?

Ta xem xét loại dữ liệu của toàn bộ các cột

In [33]:
raw_df.dtypes

Timestamp                    object
Age                           int64
Gender                       object
Country                      object
state                        object
self_employed                object
family_history               object
treatment                    object
work_interfere               object
no_employees                 object
remote_work                  object
tech_company                 object
benefits                     object
care_options                 object
wellness_program             object
seek_help                    object
anonymity                    object
leave                        object
mental_health_consequence    object
phys_health_consequence      object
coworkers                    object
supervisor                   object
mental_health_interview      object
phys_health_interview        object
mental_vs_physical           object
obs_consequence              object
comments                     object
dtype: object

Các cột có kiểu dữ liệu phù hợp. Tuy nhiên ở cột `Timestamp` ta sẽ chuyển thành kiểu `datetime`.



In [38]:
raw_df['Timestamp'] = pd.to_datetime(raw_df['Timestamp'])
raw_df.dtypes

Timestamp                    datetime64[ns]
Age                                   int64
Gender                               object
Country                              object
state                                object
self_employed                        object
family_history                       object
treatment                            object
work_interfere                       object
no_employees                         object
remote_work                          object
tech_company                         object
benefits                             object
care_options                         object
wellness_program                     object
seek_help                            object
anonymity                            object
leave                                object
mental_health_consequence            object
phys_health_consequence              object
coworkers                            object
supervisor                           object
mental_health_interview         

Các cột đã được đưa về đúng kiểu dữ liệu
### Kiểm tra sự phân bố dữ liệu các cột loại numerical

In [37]:
pd.set_option('display.float_format', lambda x : '%.2f' %x)

raw_df.drop(columns='Timestamp').describe()

,Age
count,1259.00
mean,79428148.31
std,2818299442.98
min,-1726.00
25%,27.00
50%,31.00
75%,36.00
max,99999999999.00


**Nhận xét**:
- Max của cột `Age` là một số rất lớn nằm ngoài khoảng **Tuổi** của một người. Vì vậy ta phải loại bỏ các dòng có số tuổi không đúng. 
- Min cũng tương tự.

Ta sẽ đặt khoảng giá trị của **Tuổi** từ [18, 99], bởi vì hầu hết các quốc gia chỉ định 18 tuổi là độ tuổi hợp pháp để bắt đầu làm việc.